In [ ]:
!pip3 install pytube==10.7.1
!pip3 install git+https://github.com/kazanture/YouPy.git#egg=YouPy

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
from pytube import YouTube
from pathlib import Path
import subprocess
import time

DIR_PATH = '/content/gdrive/MyDrive/YouTubeData'

df = pd.read_csv(DIR_PATH+"/youtube_data_record.csv")

In [ ]:
# To re-number files

# count = 131
# filenames = []
# for i in range (91):  
#   filenames.append("1_00"+str(count + i))

# df['Filename'] = filenames
# df.to_csv(DIR_PATH+'/youtube_data_record.csv', index=False)

In [ ]:
# To download videos in part
# new_df = df.copy(deep=True)
# new_df = new_df.loc[84:90]
# new_df = new_df.reset_index(drop=True)

In [ ]:
# Either download videos in part (30 at a time) 
# OR continuosly monitor for 429: too many reqiuests exception
# Otherwise you migth get banned from youtube
for i,link in enumerate(new_df['Fetched from']):
  try: 
    yt = YouTube(link)
    SAVE_PATH = DIR_PATH + "/webm/"
    # get the audio with highest resolution available and download
    yt.streams.filter(file_extension='webm', only_audio=True).order_by('abr').desc().first().download(SAVE_PATH, filename=new_df['Filename'][i]) # add file name while saving

    # Convert webm file format to wav
    in_name = DIR_PATH + "/webm/" + new_df['Filename'][i] + ".webm"
    out_file = DIR_PATH + "/wav/" + new_df['Filename'][i] + ".wav"
    cmd = "ffmpeg -i " + in_name + " -c:a pcm_f32le " + out_file + "  -loglevel error"
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    stdout, stderr = p.communicate()
    if p.wait()!= 0:
      print("Found some error")
      print(stderr)
    time.sleep(20)
  except Exception as e:    
    print("Caught Exception: ",e)
    print("Skipping audio file: ", new_df['Filename'][i])
    time.sleep(180)

# **YouPy Experiment**

If you start getting 429 error response either wait for retry time specified in the HTTP response header or use YouPy to add cookies and conitnue download.

NOTE: Adding personal cookies can result in account ban from YouTube

In [ ]:
import youpy as yp

In [ ]:
from YouPy import YouTubeItem

In [ ]:
video_url = 'https://www.youtube.com/watch?v=KAuecX_971A'
filename = '1_00179'
COOKIE = 'Get your cookie from browser developer tools'

In [ ]:
youtube_item = YouTubeItem(video_url, request_headers = {'cookie': COOKIE})
SAVE_PATH = DIR_PATH + "/webm/"
v_1080p = youtube_item.streams.filter(file_extension='webm', only_audio=True).order_by('abr').desc().first().download(SAVE_PATH, filename=filename)